<a href="https://colab.research.google.com/github/croissant1001/davis/blob/main/newGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import time
import pandas as pd
import math
import matplotlib.pyplot as plt
from LGA import LocalGeneticAlgorithm
import collections

class OrderedSet(collections.MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]         # sentinel node for doubly linked list
        self.map = {}                   # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)
  


class SimpleGeneticAlgorithm():
    def __init__(self,DMat,start_point):
        self.start_point = start_point
        self.item_num = len(DMat[0])
        self.left_parts = [i for i in range(self.item_num)]
        self.left_parts.remove(self.start_point)
        self.DMat = DMat
        self.seed_num = 1000
        self.offspring_num = int(0.7*self.seed_num)
        self.macro_alpha = 0.9
        self.micro_alpha = 0.1
        self.island_num=5
        self.numIters = 1000
        self.topkp = 70
        self.topko = 200
        self.elite_nump = 50
        self.elite_numo = 150

    def InitializeProcess(self):
        population = []
        for island in range(self.island_num):
            for idx in range(round(self.seed_num/self.island_num)):
                temp_list = self.left_parts[:]
                temp_occu = []
                while len(temp_list)>0:
                    t_num = random.choice(temp_list)
                    temp_occu.append(t_num)
                    temp_list.remove(t_num)
                temp_occul=[]
                temp_occul.append(temp_occu)
                temp_occul.append([island])
                population.append(temp_occul)
        return population
        
    def Fitness(self,population,record_values_dict):
        record_lists = []
        #print(population)
        for individual in population:
            key_ = ' '.join([str(i) for i in [0]+individual[0]+[0]])
            if key_ in list(record_values_dict.keys()):
                value = record_values_dict[key_]
            else:  
                #print(individual[0])
                #type(individual[0][0])
                value0 = self.DMat[self.start_point][individual[0][0]]
                #print(individual[0][0])
                for idx,item in enumerate(individual[0][:-1]):
                    value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                value0 += self.DMat[individual[0][-1]][self.start_point]
                value=[value0,individual[1]]
                # print (key_,value)
                record_values_dict[key_] = value
            record_lists.append([individual[0],1/np.log(value[0]),value[1]])
        # print (record_lists)
        sum_value = sum([i[1] for i in record_lists])
        roulette_wheel_list = []
        cumm_prob = 0
        for item in record_lists:
            cumm_prob_0 = cumm_prob
            prob_ = item[1]/sum_value
            cumm_prob += prob_
            roulette_wheel_list.append([item[0],[cumm_prob_0,cumm_prob],item[2]])
        return roulette_wheel_list,record_values_dict

    ### selecting parents
    def RouletteCrossOver(self,roulette_wheel_list):
        offsprings = []
        for turn in range(self.offspring_num):
            select_items = []
            for double in range(2):
                decision_prob = random.uniform(0,1)
                for item in roulette_wheel_list:
                    if decision_prob>=item[1][0] and decision_prob<item[1][1]:
                        select_items.append([item[0],item[2]])
            decision_orders = random.sample(range(0,self.item_num), 2)
            decision_orders.sort()
            while (decision_orders[0] == decision_orders[1]) or (decision_orders[0] <= 1 and  decision_orders[1] >= self.item_num-2):
                decision_orders = random.sample(range(0,self.item_num), 2)
                decision_orders.sort()
            new_items = []
            #print(select_items)
            # print ('parents: ',select_items,'\n','decision_orders:',decision_orders)
            for item_idx,new_item in enumerate(select_items):
                stable_part = new_item[0][decision_orders[0]:decision_orders[1]+1]
                original_length = len(stable_part)
                # non_stable_part = new_item[:decision_orders[0]][:]+ new_item[decision_orders[1]+1:][:]  
                pointer = decision_orders[1]+1
                if pointer <= len(self.left_parts[:])-1 or len(stable_part) == original_length:
                    acco_index = select_items[1-item_idx][0].index(stable_part[-1])
                else:
                    acco_index = select_items[1-item_idx][0].index(stable_part[0])
                while len(stable_part) != len(self.left_parts[:]):
                    # time.sleep(1)
                    # print ('child:',stable_part,)
                    # print ('parent1:',pointer,new_item)
                    # print ('parent2:',select_items[1-item_idx],acco_index)
                    if acco_index == len(new_item[0])-1:
                        acco_index_next = 0
                    else:
                        acco_index_next = acco_index + 1
                    if select_items[1-item_idx][0][acco_index_next] in stable_part:
                        acco_index = acco_index_next
                        continue
                    else:
                        if pointer <= len(self.left_parts[:])-1:
                            stablenumerate_part = stable_part[:] + [select_items[1-item_idx][0][acco_index_next]]
                            pointer += 1
                        else:
                            stable_part = [select_items[1-item_idx][0][acco_index_next]]+stable_part[:]
                new_prex = stable_part[:decision_orders[0]]
                new_prex.reverse()
                stable_part = new_prex[:] + stable_part[decision_orders[0]:]
                # print ('final child:',stable_part)
                new_items.append([stable_part,select_items[1-item_idx][1]])
                if len(set(new_items[item_idx][0])) != len(new_items[item_idx][0]):
                    print (new_items[item_idx])
                    print ('wrong in path!')
                    exit()
            # print ('children: ',new_items)
            offsprings.extend(new_items)
            #print(offsprings)
        return offsprings
    

    def Mutation(self,population):
        new_population = []
        for individual in population:
            decision_prob = random.uniform(0,1)
            if decision_prob >= self.macro_alpha:
                continue
            new_population.append([individual[0][::-1],individual[1]])
        for individual in population:
            new_individual = individual[0][:]
            for sub_ in new_individual:
                decision_prob = random.uniform(0,1)
                if decision_prob >= self.micro_alpha:
                    continue
                i=new_individual.index(sub_)
                j=i
                while j == i:
                    j=new_individual.index(random.choice(new_individual))
                new_individual[i],new_individual[j]=new_individual[j],new_individual[i]
            if new_individual == individual[0]:
                continue
            new_population.append([new_individual,individual[1]])  
        new_population.extend(population)  
        return new_population

    ### cost; the selection method
    
    '''   def LSO(self, population, record_values_dict):
        for individual[0] in population:
            decision_prob = random.uniform(0,1)
            if decision_prob >= self.macro_alpha:
                continue
            record_lists = []
            key_ = ' '.join([str(i) for i in [self.start_point] + individual[0][:] + [self.start_point]])
            if key_ in record_values_dict.keys():
                value = record_values_dict[key_]
            else:
                value = self.DMat[self.start_point][individual[0][0]]
                for idx, item in enumerate(individual[0][:-1]):
                     value += self.DMat[individual[0][idx]][individual[0][idx + 1]]
                value += self.DMat[individual[0][-1]][self.start_point]
                record_values_dict[key_] = value
            record_lists.append([individual[0], value])
            new_individual[0] = individual[0][:]
            better_individual[0]=individual[0][:]
            for sub_ in new_individual[0][2:]:
                i=new_individual[0].index(sub_)
                new_individual[0][1], new_individual[0][i] = new_individual[0][i], new_individual[0][1]
                valuenew = self.DMat[self.start_point][new_individual[0][0]]
                for idx, item in enumerate(new_individual[0][:-1]):
                     valuenew += self.DMat[individual[0][idx]][new_individual[0][idx + 1]]
                valuenew += self.DMat[new_individual[0][-1]][self.start_point]
                if valuenew < value:
                     better_individual[0]=new_individual[0]
                     break
            key__ = ' '.join([str(i) for i in [self.start_point] + better_individual[0][:] + [self.start_point]])
            record_values_dict[key__] = valuenew
            record_lists.append([better_individual[0], valuenew])
            individual[0]=better_individual[0]
        return population '''


    '''def Elimination(self,population,offsprings,record_values_dict):
        selected_population = []
        total_record_list = []
        total_values = []
        #print(population_island1)
        for island in range(self.island_num):
            for time in range(self.tournament_timesp):
                #print(tourn_num)
                record_lists = []
                population_island=[x for x in population if x[1]==[island]]
                #print(len(population_island))
                #competing_parents=random.sample(population_island,self.tournament_timesp*self.tournament_nump)
                #print(population_island)
                competing_group = random.sample(population_island,self.tournament_timesp if len(population_island) > self.tournament_timesp else len(population_island))
                #random.sample(population_island,self.tournament_num)
                if len(competing_group) == 0:
                # print (self.tournament_num*self.tournament_times,len(competing_individual[0]s))
                   break
                for individual in competing_group:
                    key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                    if key_ in record_values_dict.keys():
                        value = record_values_dict[key_]
                    else:
                        value0 = self.DMat[self.start_point][individual[0][0]]
                        for idx,item in enumerate(individual[0][:-1]):
                            value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                        value0 += self.DMat[individual[0][-1]][self.start_point]
                        value=[value0,individual[1]]
                        record_values_dict[key_] = value
                    record_lists.append([individual[0],value])
                record_lists.sort(key=lambda x:x[1][0])
                #print(len(record_lists))
                #print(record_lists[0])
                total_record_list.append(record_lists[0])
                selected_population.append([record_lists[0][0],record_lists[0][1][1]])
                #print(selected_population[-1])
                total_values.append(record_lists[0][1][0])
            for time in range(self.tournament_timeso):
                #print(tourn_num)
                record_lists = []
                offsprings_island=[x for x in offsprings if x[1]==[island]]
                #print(len(population_island))
                #competing_offsprings=random.sample(offsprings,self.tournament_timeso*self.tournament_numo)
                competing_group = random.sample(offsprings_island,self.tournament_timeso if len(offsprings_island) > self.tournament_timeso else len(offsprings_island))
                    # print (self.tournament_num*self.tournament_times,len(competing_individual[0]s))
                if len(competing_group) == 0:
                    break
                for individual in competing_group:
                    key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                    if key_ in record_values_dict.keys():
                        value = record_values_dict[key_]
                    else:
                        value0 = self.DMat[self.start_point][individual[0][0]]
                        for idx,item in enumerate(individual[0][:-1]):
                            value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                        value0 += self.DMat[individual[0][-1]][self.start_point]
                        value=[value0,individual[1]]
                        record_values_dict[key_] = value
                    record_lists.append([individual[0],value])
                record_lists.sort(key=lambda x:x[1][0])
                total_record_list.append(record_lists[0])
                #print(record_lists[2])
                selected_population.append([record_lists[0][0],record_lists[0][1][1]])
                total_values.append(record_lists[0][1][0])
        # print (record_lists[:10])
        diversities = []
        for i in selected_population:
            if i not in diversities:
                diversities.append(i)
            else:
                continue
        diversity_num = len(diversities)
        set_record_lists = []
        total_record_list.sort(key=lambda x:x[1][0])
        for i in total_record_list[:self.seed_num]:
            i = [self.start_point]+i[0][:]+[self.start_point],i[1]
            if i in set_record_lists:
                continue
            set_record_lists.append(i)
        print (set_record_lists[:5])
        #print(total_values)
        #print(set_record_lists)
        return selected_population,record_values_dict,set_record_lists[:10],round(np.mean(total_values),4),diversity_num'''


    ###topk
    def Elimination(self,population,offsprings,record_values_dict):
        selected_population = []
        total_record_list = []
        total_values = []
        #print(population_island1)
        for island in range(self.island_num):
            total_record1=[]
            total_record2=[]
            record_listsp = []
            population_island=[x for x in population if x[1]==[island]]
            for individual in population_island:
                key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                if key_ in record_values_dict.keys():
                    value = record_values_dict[key_]
                else:
                    value0 = self.DMat[self.start_point][individual[0][0]]
                    for idx,item in enumerate(individual[0][:-1]):
                        value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                    value0 += self.DMat[individual[0][-1]][self.start_point]
                    value=[value0,individual[1]]
                    record_values_dict[key_] = value
                record_listsp.append([individual[0],value])
            record_listsp.sort(key=lambda x:x[1][0])
                #print(len(record_lists))
                #print(record_lists[0])
            total_record1.extend(record_listsp[:self.topkp])
            record_listp=random.sample(total_record1,self.elite_nump)
            total_record_list.extend(record_listp)
            #print(total_record_list)
            for i in range(self.elite_nump):
                selected_population.append([record_listsp[i][0],record_listsp[i][1][1]])
                #print(selected_population)
                total_values.append(record_listsp[i][1][0])
            record_listso = []
            offsprings_island=[x for x in offsprings if x[1]==[island]]
            for individual in offsprings_island:
                key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                if key_ in record_values_dict.keys():
                    value = record_values_dict[key_]
                else:
                    value0 = self.DMat[self.start_point][individual[0][0]]
                    for idx,item in enumerate(individual[0][:-1]):
                        value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                    value0 += self.DMat[individual[0][-1]][self.start_point]
                    value=[value0,individual[1]]
                    record_values_dict[key_] = value
                record_listso.append([individual[0],value])
            record_listso.sort(key=lambda x:x[1][0])
            print(len(record_listso))
            total_record2.extend(record_listso[:self.topko])
            record_listso=random.sample(total_record2,self.elite_numo)
            total_record_list.extend(record_listso)
            for i in range(self.elite_numo):
                selected_population.append([record_listso[i][0],record_listso[i][1][1]])
                total_values.append(record_listso[i][1][0])
        # print (record_lists[:10])
        diversities = []
        for i in selected_population:
            if i not in diversities:
                diversities.append(i)
            else:
                continue
        diversity_num = len(diversities)
        set_record_lists = []
        total_record_list.sort(key=lambda x:x[1][0])
        for i in total_record_list[:self.seed_num]:
            i = [self.start_point]+i[0][:]+[self.start_point],i[1]
            if i in set_record_lists:
                continue
            set_record_lists.append(i)
        #print (set_record_lists[:5])
        #print(total_values)
        #print(set_record_lists)
        return selected_population,record_values_dict,set_record_lists[:10],round(np.mean(total_values),4),diversity_num

    def LGASelect(self):
        i=random.randint(1,round((self.item_num*9)/10))
        j=i+round(self.item_num/10)
        #print(i)
        #print(j)
        return i,j
    
    def LGAexcute(self,best_result,population):
      for i in range(5):
            substart,subend=self.LGASelect()
            LGA=LocalGeneticAlgorithm(best_result[i*2][0],substart,subend,DistanceMatrix)
            subtour=LGA.Iteration()
            f_best_result=best_result[i*2][0][1:substart]
            b_best_result=best_result[i*2][0][subend:-1]
    #        print(type(f_best_result))
#           print(type(b_best_result))
    #        print(type(subtour))
            nn_best_result=f_best_result+subtour+b_best_result
   #         print(nn_best_result)
    #        print(population[0])
            population.append([nn_best_result,best_result[i*2][1]])
      return population

    def Iteration(self):
        population = self.InitializeProcess()
        final_results = []
        mean_results = []
        diversity_nums = []
        record_values_dict = {}
        time_start=time.time()
        for iteration in range(self.numIters):
            roulette_wheel_list,record_values_dict = self.Fitness(population,record_values_dict)
            # offsprings = self.RouletteCrossOver(roulette_wheel_list)
            offsprings = self.RouletteCrossOver(roulette_wheel_list)
            population = self.Mutation(population)
            population,record_values_dict,best_result,mean_result,diversity_num = self.Elimination(population,offsprings,record_values_dict)
            population=self.LGAexcute(best_result,population)
            diversity_nums.append(diversity_num)
            final_results.append(round(best_result[0][1][0],4))
            mean_results.append(mean_result)
            time_end=time.time()
            if time_end-time_start >= 5*60:
                print('time cost',time_end-time_start,'s')
                print ('time is up!')
                print (best_result[0])
                print (final_results)
                plt.figure(figsize=(7,4)) 
                plt.plot(final_results,'b',lw = 1.5,label = 'best ojective value')
                plt.plot(final_results,'ro') 
                plt.plot(mean_results,'g',lw = 1.5,label = 'mean ojective value')
                plt.plot(mean_results,'ro') 
                plt.grid(axis='y')
                for a,b,c in zip(range(len(mean_results)),mean_results,diversity_nums):
                    plt.text(a, b+2, c, ha='center', va= 'bottom',fontsize=9)
                plt.legend(loc = 0) 
                plt.axis('tight')
                plt.xlabel('turns')
                plt.ylabel('total distance')
                plt.title('the convergence trend; best result: '+str(round(best_result[0][1][0],4)))
                plt.show()
                print (round(best_result[0][1][0],4))
                exit()
            

if __name__ == "__main__":
    # capacity = int(input())
    # DistanceMatrix=[
    # [ 0, 3, 6, 7, 13, 2, 4, 9],
    # [ 5, 0, 2, 3, 23, 5, 7, 1],
    # [ 6, 4, 0, 2, 4, 8, 19, 1],
    # [ 3, 7, 5, 0, 2, 3, 4, 7],
    # [ 3, 7, 15, 20, 0, 4, 4, 2],
    # [ 5, 2, 5, 3, 2, 0, 4, 7],
    # [ 6, 7, 1, 8, 5, 4, 0, 2],
    # [ 3, 4, 5, 4, 2, 4, 10, 0]]
    
    df = pd.read_csv('tour250.csv',header=None)
     # df = pd.read_csv(filename,header=None)
    DistanceMatrix = []
    for i in range(df.shape[0]):
        row = []
        for j in range(df.shape[1]):
            if df.iloc[i,j] == float("inf"):
                row.append(1e10)
            else:
                row.append(df.iloc[i,j])
            # row.append(df.iloc[i,j])
        DistanceMatrix.append(row)
    
    start_point = 0
    EA = SimpleGeneticAlgorithm(DistanceMatrix,start_point)
    SimpleGeneticAlgorithm.Iteration(EA)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  # Remove the CWD from sys.path while we load stuff.


310
286
273
274
257
294
298
273
260
270
298
276
257
283
276
281
273
273
287
278
261
289
289
303
251
285
295
288
256
270
268
279
265
291
288
291
274
282
280
268


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import random
import time
import pandas as pd
import math
import matplotlib.pyplot as plt
from LGA import LocalGeneticAlgorithm

class SimpleGeneticAlgorithm():
    def __init__(self,DMat,start_point):
        self.start_point = start_point
        self.item_num = len(DMat[0])
        self.left_parts = [i for i in range(self.item_num)]
        self.left_parts.remove(self.start_point)
        self.DMat = DMat
        self.seed_num = 1000
        self.offspring_num = int(0.6*self.seed_num)
        self.macro_alpha = 0.8
        self.micro_alpha = 0.15
        self.island_num=5
        self.numIters = 1000
        self.tournament_timesp = 80
        self.tournament_timeso = 120
        self.elite_nump = 50
        self.elite_numo = 150

    def InitializeProcess(self):
        population = []
        for island in range(self.island_num):
            for idx in range(round(self.seed_num/self.island_num)):
                temp_list = self.left_parts[:]
                temp_occu = []
                while len(temp_list)>0:
                    t_num = random.choice(temp_list)
                    temp_occu.append(t_num)
                    temp_list.remove(t_num)
                temp_occul=[]
                temp_occul.append(temp_occu)
                temp_occul.append([island])
                population.append(temp_occul)
        return population
        
    def Fitness(self,population,record_values_dict):
        record_lists = []
        #print(population)
        for individual in population:
            key_ = ' '.join([str(i) for i in [0]+individual[0]+[0]])
            if key_ in list(record_values_dict.keys()):
                value = record_values_dict[key_]
            else:  
                #print(individual[0])
                #type(individual[0][0])
                value0 = self.DMat[self.start_point][individual[0][0]]
                #print(individual[0][0])
                for idx,item in enumerate(individual[0][:-1]):
                    value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                value0 += self.DMat[individual[0][-1]][self.start_point]
                value=[value0,individual[1]]
                # print (key_,value)
                record_values_dict[key_] = value
            record_lists.append([individual[0],1/np.log(value[0]),value[1]])
        # print (record_lists)
        sum_value = sum([i[1] for i in record_lists])
        roulette_wheel_list = []
        cumm_prob = 0
        for item in record_lists:
            cumm_prob_0 = cumm_prob
            prob_ = item[1]/sum_value
            cumm_prob += prob_
            roulette_wheel_list.append([item[0],[cumm_prob_0,cumm_prob],item[2]])
        return roulette_wheel_list,record_values_dict

    ### selecting parents
    def RouletteCrossOver(self,roulette_wheel_list):
        offsprings = []
        for turn in range(self.offspring_num):
            select_items = []
            for double in range(2):
                decision_prob = random.uniform(0,1)
                for item in roulette_wheel_list:
                    if decision_prob>=item[1][0] and decision_prob<item[1][1]:
                        select_items.append([item[0],item[2]])
            decision_orders = random.sample(range(0,self.item_num), 2)
            decision_orders.sort()
            while (decision_orders[0] == decision_orders[1]) or (decision_orders[0] <= 1 and  decision_orders[1] >= self.item_num-2):
                decision_orders = random.sample(range(0,self.item_num), 2)
                decision_orders.sort()
            new_items = []
            #print(select_items)
            # print ('parents: ',select_items,'\n','decision_orders:',decision_orders)
            for item_idx,new_item in enumerate(select_items):
                stable_part = new_item[0][decision_orders[0]:decision_orders[1]+1]
                original_length = len(stable_part)
                # non_stable_part = new_item[:decision_orders[0]][:]+ new_item[decision_orders[1]+1:][:]  
                pointer = decision_orders[1]+1
                if pointer <= len(self.left_parts[:])-1 or len(stable_part) == original_length:
                    acco_index = select_items[1-item_idx][0].index(stable_part[-1])
                else:
                    acco_index = select_items[1-item_idx][0].index(stable_part[0])
                while len(stable_part) != len(self.left_parts[:]):
                    # time.sleep(1)
                    # print ('child:',stable_part,)
                    # print ('parent1:',pointer,new_item)
                    # print ('parent2:',select_items[1-item_idx],acco_index)
                    if acco_index == len(new_item[0])-1:
                        acco_index_next = 0
                    else:
                        acco_index_next = acco_index + 1
                    if select_items[1-item_idx][0][acco_index_next] in stable_part:
                        acco_index = acco_index_next
                        continue
                    else:
                        if pointer <= len(self.left_parts[:])-1:
                            stablenumerate_part = stable_part[:] + [select_items[1-item_idx][0][acco_index_next]]
                            pointer += 1
                        else:
                            stable_part = [select_items[1-item_idx][0][acco_index_next]]+stable_part[:]
                new_prex = stable_part[:decision_orders[0]]
                new_prex.reverse()
                stable_part = new_prex[:] + stable_part[decision_orders[0]:]
                # print ('final child:',stable_part)
                new_items.append([stable_part,select_items[1-item_idx][1]])
                if len(set(new_items[item_idx][0])) != len(new_items[item_idx][0]):
                    print (new_items[item_idx])
                    print ('wrong in path!')
                    exit()
            # print ('children: ',new_items)
            offsprings.extend(new_items)
            #print(offsprings)
        return offsprings
    

    def Mutation(self,population):
        new_population = []
        for individual in population:
            decision_prob = random.uniform(0,1)
            if decision_prob >= self.macro_alpha:
                continue
            new_population.append([individual[0][::-1],individual[1]])
        for individual in population:
            new_individual = individual[0][:]
            for sub_ in new_individual:
                decision_prob = random.uniform(0,1)
                if decision_prob >= self.micro_alpha:
                    continue
                i=new_individual.index(sub_)
                j=i
                while j == i:
                    j=new_individual.index(random.choice(new_individual))
                new_individual[i],new_individual[j]=new_individual[j],new_individual[i]
            if new_individual == individual[0]:
                continue
            new_population.append([new_individual,individual[1]])  
        new_population.extend(population)  
        return new_population

    ### cost; the selection method
    
    '''   def LSO(self, population, record_values_dict):
        for individual[0] in population:
            decision_prob = random.uniform(0,1)
            if decision_prob >= self.macro_alpha:
                continue
            record_lists = []
            key_ = ' '.join([str(i) for i in [self.start_point] + individual[0][:] + [self.start_point]])
            if key_ in record_values_dict.keys():
                value = record_values_dict[key_]
            else:
                value = self.DMat[self.start_point][individual[0][0]]
                for idx, item in enumerate(individual[0][:-1]):
                     value += self.DMat[individual[0][idx]][individual[0][idx + 1]]
                value += self.DMat[individual[0][-1]][self.start_point]
                record_values_dict[key_] = value
            record_lists.append([individual[0], value])
            new_individual[0] = individual[0][:]
            better_individual[0]=individual[0][:]
            for sub_ in new_individual[0][2:]:
                i=new_individual[0].index(sub_)
                new_individual[0][1], new_individual[0][i] = new_individual[0][i], new_individual[0][1]
                valuenew = self.DMat[self.start_point][new_individual[0][0]]
                for idx, item in enumerate(new_individual[0][:-1]):
                     valuenew += self.DMat[individual[0][idx]][new_individual[0][idx + 1]]
                valuenew += self.DMat[new_individual[0][-1]][self.start_point]
                if valuenew < value:
                     better_individual[0]=new_individual[0]
                     break
            key__ = ' '.join([str(i) for i in [self.start_point] + better_individual[0][:] + [self.start_point]])
            record_values_dict[key__] = valuenew
            record_lists.append([better_individual[0], valuenew])
            individual[0]=better_individual[0]
        return population '''


    '''def Elimination(self,population,offsprings,record_values_dict):
        selected_population = []
        total_record_list = []
        total_values = []
        #print(population_island1)
        for island in range(self.island_num):
            for time in range(self.tournament_timesp):
                #print(tourn_num)
                record_lists = []
                population_island=[x for x in population if x[1]==[island]]
                #print(len(population_island))
                #competing_parents=random.sample(population_island,self.tournament_timesp*self.tournament_nump)
                #print(population_island)
                competing_group = random.sample(population_island,self.tournament_timesp if len(population_island) > self.tournament_timesp else len(population_island))
                #random.sample(population_island,self.tournament_num)
                if len(competing_group) == 0:
                # print (self.tournament_num*self.tournament_times,len(competing_individual[0]s))
                   break
                for individual in competing_group:
                    key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                    if key_ in record_values_dict.keys():
                        value = record_values_dict[key_]
                    else:
                        value0 = self.DMat[self.start_point][individual[0][0]]
                        for idx,item in enumerate(individual[0][:-1]):
                            value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                        value0 += self.DMat[individual[0][-1]][self.start_point]
                        value=[value0,individual[1]]
                        record_values_dict[key_] = value
                    record_lists.append([individual[0],value])
                record_lists.sort(key=lambda x:x[1][0])
                #print(len(record_lists))
                #print(record_lists[0])
                total_record_list.append(record_lists[0])
                selected_population.append([record_lists[0][0],record_lists[0][1][1]])
                #print(selected_population[-1])
                total_values.append(record_lists[0][1][0])
            for time in range(self.tournament_timeso):
                #print(tourn_num)
                record_lists = []
                offsprings_island=[x for x in offsprings if x[1]==[island]]
                #print(len(population_island))
                #competing_offsprings=random.sample(offsprings,self.tournament_timeso*self.tournament_numo)
                competing_group = random.sample(offsprings_island,self.tournament_timeso if len(offsprings_island) > self.tournament_timeso else len(offsprings_island))
                    # print (self.tournament_num*self.tournament_times,len(competing_individual[0]s))
                if len(competing_group) == 0:
                    break
                for individual in competing_group:
                    key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                    if key_ in record_values_dict.keys():
                        value = record_values_dict[key_]
                    else:
                        value0 = self.DMat[self.start_point][individual[0][0]]
                        for idx,item in enumerate(individual[0][:-1]):
                            value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                        value0 += self.DMat[individual[0][-1]][self.start_point]
                        value=[value0,individual[1]]
                        record_values_dict[key_] = value
                    record_lists.append([individual[0],value])
                record_lists.sort(key=lambda x:x[1][0])
                total_record_list.append(record_lists[0])
                #print(record_lists[2])
                selected_population.append([record_lists[0][0],record_lists[0][1][1]])
                total_values.append(record_lists[0][1][0])
        # print (record_lists[:10])
        diversities = []
        for i in selected_population:
            if i not in diversities:
                diversities.append(i)
            else:
                continue
        diversity_num = len(diversities)
        set_record_lists = []
        total_record_list.sort(key=lambda x:x[1][0])
        for i in total_record_list[:self.seed_num]:
            i = [self.start_point]+i[0][:]+[self.start_point],i[1]
            if i in set_record_lists:
                continue
            set_record_lists.append(i)
        print (set_record_lists[:5])
        #print(total_values)
        #print(set_record_lists)
        return selected_population,record_values_dict,set_record_lists[:10],round(np.mean(total_values),4),diversity_num'''



    def Elimination(self,population,offsprings,record_values_dict):
        selected_population = []
        total_record_list = []
        total_values = []
        #print(population_island1)
        for island in range(self.island_num):
            record_listsp = []
            population_island=[x for x in population if x[1]==[island]]
            for individual in population_island:
                key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                if key_ in record_values_dict.keys():
                    value = record_values_dict[key_]
                else:
                    value0 = self.DMat[self.start_point][individual[0][0]]
                    for idx,item in enumerate(individual[0][:-1]):
                        value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                    value0 += self.DMat[individual[0][-1]][self.start_point]
                    value=[value0,individual[1]]
                    record_values_dict[key_] = value
                record_listsp.append([individual[0],value])
            record_listsp.sort(key=lambda x:x[1][0])
                #print(len(record_lists))
                #print(record_lists[0])
            total_record_list.extend(record_listsp[:self.elite_nump])
            #print(total_record_list)
            for i in range(self.elite_nump):
                selected_population.append([record_listsp[i][0],record_listsp[i][1][1]])
                #print(selected_population)
                total_values.append(record_listsp[i][1][0])
            record_listso = []
            offsprings_island=[x for x in offsprings if x[1]==[island]]
            for individual in offsprings_island:
                key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                if key_ in record_values_dict.keys():
                    value = record_values_dict[key_]
                else:
                    value0 = self.DMat[self.start_point][individual[0][0]]
                    for idx,item in enumerate(individual[0][:-1]):
                        value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                    value0 += self.DMat[individual[0][-1]][self.start_point]
                    value=[value0,individual[1]]
                    record_values_dict[key_] = value
                record_listso.append([individual[0],value])
            record_listso.sort(key=lambda x:x[1][0])
            print(len(record_listso))
            total_record_list.extend(record_listso[:self.elite_numo])
            for i in range(self.elite_numo):
                selected_population.append([record_listso[i][0],record_listso[i][1][1]])
                total_values.append(record_listso[i][1][0])
        # print (record_lists[:10])
        diversities = []
        for i in selected_population:
            if i not in diversities:
                diversities.append(i)
            else:
                continue
        diversity_num = len(diversities)
        set_record_lists = []
        total_record_list.sort(key=lambda x:x[1][0])
        for i in total_record_list[:self.seed_num]:
            i = [self.start_point]+i[0][:]+[self.start_point],i[1]
            if i in set_record_lists:
                continue
            set_record_lists.append(i)
        #print (set_record_lists[:5])
        #print(total_values)
        #print(set_record_lists)
        return selected_population,record_values_dict,set_record_lists[:10],round(np.mean(total_values),4),diversity_num

    def LGASelect(self):
        i=random.randint(1,round((self.item_num*9)/10))
        j=i+round(self.item_num/10)
        #print(i)
        #print(j)
        return i,j
    
    def LGAexcute(self,best_result,population):
      for i in range(5):
            substart,subend=self.LGASelect()
            LGA=LocalGeneticAlgorithm(best_result[i*2][0],substart,subend,DistanceMatrix)
            subtour=LGA.Iteration()
            f_best_result=best_result[i*2][0][1:substart]
            b_best_result=best_result[i*2][0][subend:-1]
    #        print(type(f_best_result))
#           print(type(b_best_result))
    #        print(type(subtour))
            nn_best_result=f_best_result+subtour+b_best_result
   #         print(nn_best_result)
    #        print(population[0])
            population.append([nn_best_result,best_result[i*2][1]])
      return population

    def Iteration(self):
        population = self.InitializeProcess()
        final_results = []
        mean_results = []
        diversity_nums = []
        record_values_dict = {}
        time_start=time.time()
        for iteration in range(self.numIters):
            roulette_wheel_list,record_values_dict = self.Fitness(population,record_values_dict)
            # offsprings = self.RouletteCrossOver(roulette_wheel_list)
            offsprings = self.RouletteCrossOver(roulette_wheel_list)
            population = self.Mutation(population)
            population,record_values_dict,best_result,mean_result,diversity_num = self.Elimination(population,offsprings,record_values_dict)
            population=self.LGAexcute(best_result,population)
            diversity_nums.append(diversity_num)
            final_results.append(round(best_result[0][1][0],4))
            mean_results.append(mean_result)
            time_end=time.time()
            if time_end-time_start >= 5*60:
                print('time cost',time_end-time_start,'s')
                print ('time is up!')
                print (best_result[0])
                print (final_results)
                plt.figure(figsize=(7,4)) 
                plt.plot(final_results,'b',lw = 1.5,label = 'best ojective value')
                plt.plot(final_results,'ro') 
                plt.plot(mean_results,'g',lw = 1.5,label = 'mean ojective value')
                plt.plot(mean_results,'ro') 
                plt.grid(axis='y')
                for a,b,c in zip(range(len(mean_results)),mean_results,diversity_nums):
                    plt.text(a, b+2, c, ha='center', va= 'bottom',fontsize=9)
                plt.legend(loc = 0) 
                plt.axis('tight')
                plt.xlabel('turns')
                plt.ylabel('total distance')
                plt.title('the convergence trend; best result: '+str(round(best_result[0][1][0],4)))
                plt.show()
                print (round(best_result[0][1][0],4))
                exit()
            

if __name__ == "__main__":
    # capacity = int(input())
    # DistanceMatrix=[
    # [ 0, 3, 6, 7, 13, 2, 4, 9],
    # [ 5, 0, 2, 3, 23, 5, 7, 1],
    # [ 6, 4, 0, 2, 4, 8, 19, 1],
    # [ 3, 7, 5, 0, 2, 3, 4, 7],
    # [ 3, 7, 15, 20, 0, 4, 4, 2],
    # [ 5, 2, 5, 3, 2, 0, 4, 7],
    # [ 6, 7, 1, 8, 5, 4, 0, 2],
    # [ 3, 4, 5, 4, 2, 4, 10, 0]]
    
    df = pd.read_csv('tour250.csv',header=None)
     # df = pd.read_csv(filename,header=None)
    DistanceMatrix = []
    for i in range(df.shape[0]):
        row = []
        for j in range(df.shape[1]):
            if df.iloc[i,j] == float("inf"):
                row.append(1e10)
            else:
                row.append(df.iloc[i,j])
            # row.append(df.iloc[i,j])
        DistanceMatrix.append(row)
    
    start_point = 0
    EA = SimpleGeneticAlgorithm(DistanceMatrix,start_point)
    SimpleGeneticAlgorithm.Iteration(EA)

↓ 改了recombination

In [ ]:
import numpy as np
import random
import time
import pandas as pd
import math
import matplotlib.pyplot as plt
from LGA import LocalGeneticAlgorithm
import collections


class OrderedSet(collections.MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]         # sentinel node for doubly linked list
        self.map = {}                   # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)
  



class SimpleGeneticAlgorithm():
    def __init__(self,DMat,start_point):
        self.start_point = start_point
        self.item_num = len(DMat[0])
        self.left_parts = [i for i in range(self.item_num)]
        self.left_parts.remove(self.start_point)
        self.DMat = DMat
        self.seed_num = 1000
        self.offspring_num = int(0.8*self.seed_num)
        self.macro_alpha = 0.8
        self.micro_alpha = 0.15
        self.island_num=5
        self.numIters = 1000
        self.tournament_timesp = 80
        self.tournament_timeso = 120
        self.tournament_nump=30
        self.tournament_numo=30
        self.elite_nump = 50
        self.elite_numo = 150

    def InitializeProcess(self):
        population = []
        for island in range(self.island_num):
            for idx in range(round(self.seed_num/self.island_num)):
                temp_list = self.left_parts[:]
                temp_occu = []
                while len(temp_list)>0:
                    t_num = random.choice(temp_list)
                    temp_occu.append(t_num)
                    temp_list.remove(t_num)
                temp_occul=[]
                temp_occul.append(temp_occu)
                temp_occul.append([island])
                population.append(temp_occul)
        return population
        
    def Fitness(self,population,record_values_dict):
        record_lists = []
        #print(population)
        for individual in population:
            key_ = ' '.join([str(i) for i in [0]+individual[0]+[0]])
            if key_ in list(record_values_dict.keys()):
                value = record_values_dict[key_]
            else:  
                #print(individual[0])
                #type(individual[0][0])
                value0 = self.DMat[self.start_point][individual[0][0]]
                #print(individual[0][0])
                for idx,item in enumerate(individual[0][:-1]):
                    value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                value0 += self.DMat[individual[0][-1]][self.start_point]
                value=[value0,individual[1]]
                # print (key_,value)
                record_values_dict[key_] = value
            record_lists.append([individual[0],1/np.log(value[0]),value[1]])
        # print (record_lists)
        sum_value = sum([i[1] for i in record_lists])
        roulette_wheel_list = []
        cumm_prob = 0
        for item in record_lists:
            cumm_prob_0 = cumm_prob
            prob_ = item[1]/sum_value
            cumm_prob += prob_
            roulette_wheel_list.append([item[0],[cumm_prob_0,cumm_prob],item[2]])
        return roulette_wheel_list,record_values_dict

    ### selecting parents
    def RouletteCrossOver(self,roulette_wheel_list):
        offsprings = []
        for turn in range(self.offspring_num):
            select_items = []
            for double in range(2):
                decision_prob = random.uniform(0,1)
                for item in roulette_wheel_list:
                    if decision_prob>=item[1][0] and decision_prob<item[1][1]:
                        select_items.append([item[0],item[2]])
            decision_orders = random.sample(range(0,self.item_num), 2)
            decision_orders.sort()
            while (decision_orders[0] == decision_orders[1]) or (decision_orders[0] <= 1 and  decision_orders[1] >= self.item_num-2):
                decision_orders = random.sample(range(0,self.item_num), 2)
                decision_orders.sort()
            new_items = []
            #print(select_items)
            # print ('parents: ',select_items,'\n','decision_orders:',decision_orders)
            for item_idx,new_item in enumerate(select_items):
                stable_part = new_item[0][decision_orders[0]:decision_orders[1]+1]
                original_length = len(stable_part)
                # non_stable_part = new_item[:decision_orders[0]][:]+ new_item[decision_orders[1]+1:][:]  
                for e in (OrderedSet(select_items[1-item_idx][0])-OrderedSet(stable_part)):
                  stable_part.append(e)
                new_items.append([stable_part,new_item[1]])
            #print ('children: ',new_items)
            offsprings.extend(new_items)
            #print(offsprings)
        return offsprings
    

    def Mutation(self,population):
        new_population = []
        for individual in population:
            decision_prob = random.uniform(0,1)
            if decision_prob >= self.macro_alpha:
                continue
            new_population.append([individual[0][::-1],individual[1]])
        for individual in population:
            new_individual = individual[0][:]
            for sub_ in new_individual:
                decision_prob = random.uniform(0,1)
                if decision_prob >= self.micro_alpha:
                    continue
                i=new_individual.index(sub_)
                j=i
                while j == i:
                    j=new_individual.index(random.choice(new_individual))
                new_individual[i],new_individual[j]=new_individual[j],new_individual[i]
            if new_individual == individual[0]:
                continue
            new_population.append([new_individual,individual[1]])  
        new_population.extend(population)  
        return new_population

    ### cost; the selection method
    
    '''   def LSO(self, population, record_values_dict):
        for individual[0] in population:
            decision_prob = random.uniform(0,1)
            if decision_prob >= self.macro_alpha:
                continue
            record_lists = []
            key_ = ' '.join([str(i) for i in [self.start_point] + individual[0][:] + [self.start_point]])
            if key_ in record_values_dict.keys():
                value = record_values_dict[key_]
            else:
                value = self.DMat[self.start_point][individual[0][0]]
                for idx, item in enumerate(individual[0][:-1]):
                     value += self.DMat[individual[0][idx]][individual[0][idx + 1]]
                value += self.DMat[individual[0][-1]][self.start_point]
                record_values_dict[key_] = value
            record_lists.append([individual[0], value])
            new_individual[0] = individual[0][:]
            better_individual[0]=individual[0][:]
            for sub_ in new_individual[0][2:]:
                i=new_individual[0].index(sub_)
                new_individual[0][1], new_individual[0][i] = new_individual[0][i], new_individual[0][1]
                valuenew = self.DMat[self.start_point][new_individual[0][0]]
                for idx, item in enumerate(new_individual[0][:-1]):
                     valuenew += self.DMat[individual[0][idx]][new_individual[0][idx + 1]]
                valuenew += self.DMat[new_individual[0][-1]][self.start_point]
                if valuenew < value:
                     better_individual[0]=new_individual[0]
                     break
            key__ = ' '.join([str(i) for i in [self.start_point] + better_individual[0][:] + [self.start_point]])
            record_values_dict[key__] = valuenew
            record_lists.append([better_individual[0], valuenew])
            individual[0]=better_individual[0]
        return population '''


    def Elimination(self,population,offsprings,record_values_dict):
        selected_population = []
        total_record_list = []
        total_values = []
        #print(population_island1)
        for island in range(self.island_num):
            for time in range(self.tournament_timesp):
                #print(tourn_num)
                record_lists = []
                population_island=[x for x in population if x[1]==[island]]
                #print(len(population_island))
                #competing_parents=random.sample(population_island,self.tournament_timesp*self.tournament_nump)
                #print(population_island)
                competing_group = random.sample(population_island,self.tournament_nump)
                if len(competing_group) == 0:
                # print (self.tournament_num*self.tournament_times,len(competing_individual[0]s))
                   break
                for individual in competing_group:
                    key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                    if key_ in record_values_dict.keys():
                        value = record_values_dict[key_]
                    else:
                        value0 = self.DMat[self.start_point][individual[0][0]]
                        for idx,item in enumerate(individual[0][:-1]):
                            value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                        value0 += self.DMat[individual[0][-1]][self.start_point]
                        value=[value0,individual[1]]
                        record_values_dict[key_] = value
                    record_lists.append([individual[0],value])
                record_lists.sort(key=lambda x:x[1][0])
                #print(len(record_lists))
                #print(record_lists[0])
                total_record_list.append(record_lists[0])
                selected_population.append([record_lists[0][0],record_lists[0][1][1]])
                #print(selected_population[-1])
                total_values.append(record_lists[0][1][0])
            for time in range(self.tournament_timeso):
                #print(tourn_num)
                record_lists = []
                offsprings_island=[x for x in offsprings if x[1]==[island]]
                #print(len(offsprings_island))
                #competing_offsprings=random.sample(offsprings,self.tournament_timeso*self.tournament_numo)
                competing_group = random.sample(offsprings_island,self.tournament_numo)
                    # print (self.tournament_num*self.tournament_times,len(competing_individual[0]s))
                if len(competing_group) == 0:
                    break
                for individual in competing_group:
                    key_ = ' '.join([str(i) for i in [self.start_point]+individual[0][:]+[self.start_point]])
                    if key_ in record_values_dict.keys():
                        value = record_values_dict[key_]
                    else:
                        value0 = self.DMat[self.start_point][individual[0][0]]
                        for idx,item in enumerate(individual[0][:-1]):
                            value0 += self.DMat[individual[0][idx]][individual[0][idx+1]]
                        value0 += self.DMat[individual[0][-1]][self.start_point]
                        value=[value0,individual[1]]
                        record_values_dict[key_] = value
                    record_lists.append([individual[0],value])
                record_lists.sort(key=lambda x:x[1][0])
                total_record_list.append(record_lists[0])
                #print(record_lists[2])
                selected_population.append([record_lists[0][0],record_lists[0][1][1]])
                total_values.append(record_lists[0][1][0])
        # print (record_lists[:10])
        diversities = []
        for i in selected_population:
            if i not in diversities:
                diversities.append(i)
            else:
                continue
        diversity_num = len(diversities)
        set_record_lists = []
        total_record_list.sort(key=lambda x:x[1][0])
        for i in total_record_list[:self.seed_num]:
            i = [self.start_point]+i[0][:]+[self.start_point],i[1]
            if i in set_record_lists:
                continue
            set_record_lists.append(i)
        print (set_record_lists[:5])
        #print(total_values)
        #print(set_record_lists)
        return selected_population,record_values_dict,set_record_lists[:20],round(np.mean(total_values),4),diversity_num



    def LGASelect(self):
        i=random.randint(1,round((self.item_num*9)/10))
        j=i+round(self.item_num/10)
        #print(i)
        #print(j)
        return i,j
    
    def LGAexcute(self,best_result,population):
      for i in range(5):
            substart,subend=self.LGASelect()
            LGA=LocalGeneticAlgorithm(best_result[i][0],substart,subend,DistanceMatrix)
            subtour=LGA.Iteration()
            f_best_result=best_result[i][0][1:substart]
            b_best_result=best_result[i][0][subend:-1]
    #        print(type(f_best_result))
#           print(type(b_best_result))
    #        print(type(subtour))
            nn_best_result=f_best_result+subtour+b_best_result
   #         print(nn_best_result)
    #        print(population[0])
            population.append([nn_best_result,best_result[i][1]])
      return population

    def Iteration(self):
        population = self.InitializeProcess()
        final_results = []
        mean_results = []
        diversity_nums = []
        record_values_dict = {}
        time_start=time.time()
        best_record=[]
        for iteration in range(self.numIters):
            roulette_wheel_list,record_values_dict = self.Fitness(population,record_values_dict)
            # offsprings = self.RouletteCrossOver(roulette_wheel_list)
            offsprings = self.RouletteCrossOver(roulette_wheel_list)
            population = self.Mutation(population)
            population,record_values_dict,best_result,mean_result,diversity_num = self.Elimination(population,offsprings,record_values_dict)
            be
            population=self.LGAexcute(best_result,population)
            diversity_nums.append(diversity_num)
            final_results.append(round(best_result[0][1][0],4))
            mean_results.append(mean_result)
            time_end=time.time()
            if time_end-time_start >= 5*60:
                print('time cost',time_end-time_start,'s')
                print ('time is up!')
                print (best_result[0])
                print (final_results)
                plt.figure(figsize=(7,4)) 
                plt.plot(final_results,'b',lw = 1.5,label = 'best ojective value')
                plt.plot(final_results,'ro') 
                plt.plot(mean_results,'g',lw = 1.5,label = 'mean ojective value')
                plt.plot(mean_results,'ro') 
                plt.grid(axis='y')
                for a,b,c in zip(range(len(mean_results)),mean_results,diversity_nums):
                    plt.text(a, b+2, c, ha='center', va= 'bottom',fontsize=9)
                plt.legend(loc = 0) 
                plt.axis('tight')
                plt.xlabel('turns')
                plt.ylabel('total distance')
                plt.title('the convergence trend; best result: '+str(round(best_result[0][1][0],4)))
                plt.show()
                print (round(best_result[0][1][0],4))
                exit()
            

if __name__ == "__main__":
    # capacity = int(input())
    # DistanceMatrix=[
    # [ 0, 3, 6, 7, 13, 2, 4, 9],
    # [ 5, 0, 2, 3, 23, 5, 7, 1],
    # [ 6, 4, 0, 2, 4, 8, 19, 1],
    # [ 3, 7, 5, 0, 2, 3, 4, 7],
    # [ 3, 7, 15, 20, 0, 4, 4, 2],
    # [ 5, 2, 5, 3, 2, 0, 4, 7],
    # [ 6, 7, 1, 8, 5, 4, 0, 2],
    # [ 3, 4, 5, 4, 2, 4, 10, 0]]
    
    df = pd.read_csv('tour250.csv',header=None)
     # df = pd.read_csv(filename,header=None)
    DistanceMatrix = []
    for i in range(df.shape[0]):
        row = []
        for j in range(df.shape[1]):
            if df.iloc[i,j] == float("inf"):
                row.append(1e10)
            else:
                row.append(df.iloc[i,j])
            # row.append(df.iloc[i,j])
        DistanceMatrix.append(row)
    
    start_point = 0
    EA = SimpleGeneticAlgorithm(DistanceMatrix,start_point)
    SimpleGeneticAlgorithm.Iteration(EA)




KeyboardInterrupt: ignored